In [1]:
# Import our dependencies
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import sklearn as skl
import tensorflow as tf
from pathlib import Path
import os
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
# Import our input dataset
file_path = Path('./Resources/HR-Employee-Attrition.csv')
attrition_df = pd.read_csv(file_path)
attrition_df.head()


,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [3]:
# Generate our categorical variable list
attrition_cat = attrition_df.dtypes[attrition_df.dtypes == "object"].index.tolist()
attrition_cat

['Attrition',
 'BusinessTravel',
 'Department',
 'EducationField',
 'Gender',
 'JobRole',
 'MaritalStatus',
 'Over18',
 'OverTime']

In [4]:
# Check the number of unique values in each column
attrition_df[attrition_cat].nunique()

Attrition         2
BusinessTravel    3
Department        3
EducationField    6
Gender            2
JobRole           9
MaritalStatus     3
Over18            1
OverTime          2
dtype: int64

In [5]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(attrition_df[attrition_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(attrition_cat)
encode_df.head()

,Attrition_No,Attrition_Yes,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Human Resources,Department_Research & Development,Department_Sales,EducationField_Human Resources,EducationField_Life Sciences,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


In [6]:
# Merge one-hot encoded features and drop the originals
attrition_df = attrition_df.merge(encode_df,left_index=True, right_index=True)
attrition_df = attrition_df.drop(attrition_cat,1)
attrition_df.head()


,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,41,1102,1,2,1,1,2,94,3,2,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,49,279,8,1,1,2,3,61,2,2,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,37,1373,2,2,1,4,4,92,2,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3,33,1392,3,4,1,5,4,56,3,1,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,27,591,2,1,1,7,1,40,3,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


In [7]:
# Split our preprocessed data into our features and target arrays
y = attrition_df["Attrition_Yes"].values
X = attrition_df.drop(["Attrition_Yes","Attrition_No"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)


In [8]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 448       
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 45        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 6         
Total params: 499
Trainable params: 499
Non-trainable params: 0
_________________________________________________________________


In [10]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [11]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [12]:
# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=1000)

In [13]:
# Train the model
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Train on 1102 samples
Epoch 1/100
 480/1102 [============>.................] - ETA: 1s - loss: 0.6741 - accuracy: 0.6938 
Epoch 00001: saving model to checkpoints/weights.01.hdf5
1102/1102 [==============================] - 2s 1ms/sample - loss: 0.6407 - accuracy: 0.7287
Epoch 2/100
 416/1102 [==========>...................] - ETA: 0s - loss: 0.5658 - accuracy: 0.7933
Epoch 00002: saving model to checkpoints/weights.02.hdf5
1102/1102 [==============================] - 0s 117us/sample - loss: 0.5675 - accuracy: 0.7886
Epoch 3/100
 480/1102 [============>.................] - ETA: 0s - loss: 0.5262 - accuracy: 0.8250
Epoch 00003: saving model to checkpoints/weights.03.hdf5
1102/1102 [==============================] - 0s 106us/sample - loss: 0.5169 - accuracy: 0.8158
Epoch 4/100
 576/1102 [==============>...............] - ETA: 0s - loss: 0.4589 - accuracy: 0.8438
Epoch 00004: saving model to checkpoints/weights.04.hdf5
1102/1102 [==============================] - 0s 116us/sample - loss: 0

 608/1102 [===============>..............] - ETA: 0s - loss: 0.2319 - accuracy: 0.9112
Epoch 00032: saving model to checkpoints/weights.32.hdf5
1102/1102 [==============================] - 0s 92us/sample - loss: 0.2502 - accuracy: 0.9020
Epoch 33/100
 768/1102 [===================>..........] - ETA: 0s - loss: 0.2511 - accuracy: 0.8984
Epoch 00033: saving model to checkpoints/weights.33.hdf5
1102/1102 [==============================] - 0s 80us/sample - loss: 0.2480 - accuracy: 0.9011
Epoch 34/100
 800/1102 [====================>.........] - ETA: 0s - loss: 0.2448 - accuracy: 0.9062
Epoch 00034: saving model to checkpoints/weights.34.hdf5
1102/1102 [==============================] - 0s 76us/sample - loss: 0.2463 - accuracy: 0.9029
Epoch 35/100
 800/1102 [====================>.........] - ETA: 0s - loss: 0.2718 - accuracy: 0.8913
Epoch 00035: saving model to checkpoints/weights.35.hdf5
1102/1102 [==============================] - 0s 78us/sample - loss: 0.2444 - accuracy: 0.9056
Epoch 36/

1102/1102 [==============================] - 0s 78us/sample - loss: 0.2060 - accuracy: 0.9238
Epoch 64/100
  32/1102 [..............................] - ETA: 0s - loss: 0.1732 - accuracy: 0.9062
Epoch 00064: saving model to checkpoints/weights.64.hdf5
 544/1102 [=============>................] - ETA: 0s - loss: 0.2382 - accuracy: 0.8989
Epoch 00064: saving model to checkpoints/weights.64.hdf5
1102/1102 [==============================] - 0s 90us/sample - loss: 0.2063 - accuracy: 0.9211
Epoch 65/100
 800/1102 [====================>.........] - ETA: 0s - loss: 0.2034 - accuracy: 0.9262
Epoch 00065: saving model to checkpoints/weights.65.hdf5
1102/1102 [==============================] - 0s 76us/sample - loss: 0.2046 - accuracy: 0.9256
Epoch 66/100
 832/1102 [=====================>........] - ETA: 0s - loss: 0.1901 - accuracy: 0.9291
Epoch 00066: saving model to checkpoints/weights.66.hdf5
1102/1102 [==============================] - 0s 73us/sample - loss: 0.2044 - accuracy: 0.9256
Epoch 67/

1102/1102 [==============================] - 0s 74us/sample - loss: 0.1738 - accuracy: 0.9356
Epoch 95/100
  32/1102 [..............................] - ETA: 0s - loss: 0.1136 - accuracy: 0.9375
Epoch 00095: saving model to checkpoints/weights.95.hdf5
1102/1102 [==============================] - 0s 72us/sample - loss: 0.1730 - accuracy: 0.9365
Epoch 96/100
  32/1102 [..............................] - ETA: 0s - loss: 0.1691 - accuracy: 0.9688
Epoch 00096: saving model to checkpoints/weights.96.hdf5
 736/1102 [===================>..........] - ETA: 0s - loss: 0.1724 - accuracy: 0.9348
Epoch 00096: saving model to checkpoints/weights.96.hdf5
1102/1102 [==============================] - 0s 76us/sample - loss: 0.1715 - accuracy: 0.9356
Epoch 97/100
 896/1102 [=======================>......] - ETA: 0s - loss: 0.1616 - accuracy: 0.9397
Epoch 00097: saving model to checkpoints/weights.97.hdf5
1102/1102 [==============================] - 0s 69us/sample - loss: 0.1702 - accuracy: 0.9356
Epoch 98/

In [14]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

368/1 - 0s - loss: 743.4265 - accuracy: 0.8723
Loss: 1059.7751013714335, Accuracy: 0.8722826242446899


In [15]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn_new = tf.keras.models.Sequential()

# First hidden layer
nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_new.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn_new.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Restore the model weights
nn_new.load_weights("checkpoints/weights.100.hdf5")

# Evaluate the model using the test data
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

368/1 - 0s - loss: 0.2517 - accuracy: 0.8750
Loss: 0.40972138941287994, Accuracy: 0.875


In [16]:
# Export our model to HDF5 file
nn_new.save("trained_attrition.h5")